In [ ]:
import os
import pandas as pd
import numpy as np
import json
import seaborn as sns
import funcTest as ft
import funcData as fd

In [ ]:
path_rawdata = 'F://TFG//datasets/raw_datasets//'
path_train = 'F://TFG//datasets//data_train//'
path = 'F:/TFG/datasets/nature-dataset/'


rawdata = pd.read_csv(path_rawdata+'rawDB.csv')
footdata = pd.read_csv(path_rawdata+'fdataDB.csv')
partidos_raw = pd.read_json(path_rawdata+'RAW_partidos.json')

#### Import of data

In [ ]:
tags = pd.read_csv(path+'tags2name.csv')
tags.set_index('Tag',inplace=True,drop=True)

eventosDF = pd.read_json(path_rawdata+'eventos_raw.json')
eventosDF.set_index('id',inplace=True)
teams = pd.read_csv(path_rawdata+'teams.csv',index_col='wyId',delimiter=';')
players = pd.read_csv(path_rawdata+'players.csv',index_col='wyId',delimiter=';')

eventos_dict = list(set(zip(eventosDF.eventId,eventosDF.eventName)))
eventos_dict = {v:k for k,v in eventos_dict}

In [ ]:
matches = pd.read_csv(path_rawdata+'Nature_wStatsDB_v2.csv')
matches.set_index('wyId',inplace=True)
matches[:4]

In [ ]:
shots = pd.read_json(path_rawdata+'shots.json')
passes = pd.read_json(path_rawdata+'passes.json')


In [ ]:
matches_stats = pd.read_csv(path_rawdata+'match_stats.csv',sep=';',index_col='matchId')
# matches_stats.drop(['teamId_sh1', 'teamId_sh2'],axis=1,inplace=True)
matches_stats[:3]

#### Calculo de los ultimos N partidos

In [ ]:
def ult_partidos(npj=4,nj=4):

    partidos_h = {}
    partidos_a = {}

    for mtc in matches.index.to_list():
        tid_h, tid_a = matches.loc[mtc].teamId_home, matches.loc[mtc].teamId_away
        mask_time = (matches.loc[mtc].dateutc > matches.dateutc)
        mask_team_h, mask_team_a = (matches.teamId_home == tid_h), (matches.teamId_away == tid_a)
        mask_h, mask_a = mask_time & mask_team_h, mask_time & mask_team_a
        # assert sum(mask_h) == sum(mask_a)
        # hecho para que solo coja los ultimos partidos en casa si el partido actual es en casa. Igual para visitante.

        if sum(mask_h) >= npj:
            partidos_anteriores_h = (matches[mask_h][:npj].index).to_numpy()
        else:
            partidos_anteriores_h = np.nan

        if sum(mask_a) >= npj:
            partidos_anteriores_a = (matches[mask_a][:npj].index).to_numpy()
        else:
            partidos_anteriores_a = np.nan

        

        partidos_h[mtc], partidos_a[mtc] = partidos_anteriores_h, partidos_anteriores_a

        # jugadores mas disparos

    return partidos_h, partidos_a

In [ ]:
npj = 4
partidos_h, partidos_a = ult_partidos(npj)

#### Calculo minutos de los jugadores

In [ ]:
playerIds = np.array(players.index)

In [ ]:
indice          = [] # idmatch_idteam
players_id_mins = []
players_mins    = []

# tambien estamos interesados en mantener un diccionario con todos los minutos que han jugado cada jugador y en que partidos
players_match_mins = {pId:{} for pId in playerIds}    # creamos un dict vacio para cada jugador
partidos_raw.set_index('wyId',inplace=True)

for m in matches.reset_index().wyId:
    form_info = partidos_raw.teamsData.loc[m]                                   # seleccionamos el campo de las formaciones
    for t in list(form_info.keys()):                                            # iteramos sobre los dos equipos
        indice.append((m,int(t)))                                               # guardamos en una lista de tuplas lo que seran las keys de la tabla (partido,equipo)
        form_t = form_info[t]['formation']                                      # seleccionamos la formacion de uno de los equipos
        dict_team1 = {pl['playerId']:0 for pl in form_t['bench']}              
        dict_team2 = {pl['playerId']:90 for pl in form_t['lineup']}
        dict_match = {**dict_team1, **dict_team2}
        # añadimos los minutos en el dict de los minutos de cada jugador a lo largo de la temporada
        for pl in form_t['bench']:
            if players_match_mins.get(pl['playerId'],-1) == -1: players_match_mins[pl['playerId']] = {}
            players_match_mins[pl['playerId']][m] = 0
        for pl in form_t['lineup']:
            if players_match_mins.get(pl['playerId'],-1) == -1: players_match_mins[pl['playerId']] = {}
            players_match_mins[pl['playerId']][m] = 90
        # print((m,int(t)))
        if form_t['substitutions'] != 'null':
            for subs in form_t['substitutions']:
                # print(subs['minute'])
                if players_match_mins.get(subs['playerIn'],-1) != -1 and players_match_mins.get(subs['playerOut'],-1) != -1:
                    mins = subs['minute'] if (subs['minute'] < 90) else 90
                    dict_match[subs['playerIn']] = 90-mins
                    players_match_mins[subs['playerIn']][m] = 90-mins  
                    dict_match[subs['playerOut']] = dict_match[subs['playerOut']] - (90-mins)  
                    players_match_mins[subs['playerOut']][m] = dict_match[subs['playerOut']] - (90-mins) 
                

        players_id_mins.append(np.array(list(dict_match.keys())))   
        players_mins.append(np.array(list(dict_match.values())))   

players_id_mins = np.array(players_id_mins)
players_mins = np.array(players_mins)


In [ ]:
assert len(players_id_mins) == len(players_mins) == len(indice)

In [ ]:
# todos los equipos deberian sumar 990 minutos en un partido entre sus jugadores

for mId,mins in zip(indice,players_mins):
    if np.sum(mins) != 990:
        print(mId, mins)
    # assert np.sum(m) == 11*90

# PARTIDOS QUE NO CUMPLEN LOS MINUTOS:
# (2575965, 3204) [ 0  0  0  0  0  0  0  0  0  0  0  0 56 62 88 90 90 90 90 90 90 90 90  2] -> no estan registrados los IDs de los jugadores que entran al partido
# (2575959, 3158) [ 0  0  0  0  0  0  0  0  0  0  0 74 81 84 90 90 90 90 90 90 90 90  6] -> no estan registrados los IDs de los jugadores que entran al partido

In [ ]:
indice_list = np.array(indice)

# vamos a crear un dataframe con dichos arrays

players_mins_DF = pd.DataFrame({'matchesId': indice_list[:,0], 'teamsId': indice_list[:,1], 'players_id': players_id_mins, 'players_min': players_mins})
players_mins_DF.set_index(['matchesId','teamsId'],drop=True,inplace=True)
players_mins_DF

### Vector de caracteristicas para cada equipo teniendo solo en cuenta el LineUp

- Features defensivas -> tackles, fouls, interception, clearances...
- Features portero -> saves attempts
- Features ataque -> smart passes, cross passes, duels
- Features generales -> missed balls, time loss,  

Vamos a crear estadisticas de cada equipo en cada partido, partiendo de los jugadores del lineup.

Evaluaremos:
- Estadisticas como valor absoluto -> i.e. # smart passes
- Estadisticas por minutos jugados (sin weighted) -> i.e. # smart passes / # minutos jugados
- Estadisticas por minutos jugados (weighted) -> i.e. # smart passes * # minutos jugados / # minutos jugados

#### Vector minutos x equipo

In [ ]:
aux1 = np.array([0,1,2,3,4,5,6])
aux2 = np.array([3,4,6,9,8])
np.intersect1d(aux1,aux2)

In [ ]:
# seleccionamos solo los partidos que tienen 4 partidos anteriores
idxs_h = np.array(pd.DataFrame(partidos_h).T.reset_index().dropna(axis=0)['index'].to_list())
idxs_a = np.array(pd.DataFrame(partidos_a).T.reset_index().dropna(axis=0)['index'].to_list())
idxs   = np.intersect1d(idxs_h,idxs_a)

matches_wUltPartidos = matches_stats.set_index('wyId').loc[idxs]
matches_wUltPartidos.shape

In [ ]:
matches_wUltPartidos.columns

In [ ]:
# iteramos sobre cada partido de matches_stats y para cada equipo seleccionamos los jugadores titulares.

minsH, minsA = np.zeros(len(matches_wUltPartidos)), np.zeros(len(matches_wUltPartidos))

for i,m in enumerate(matches_wUltPartidos.reset_index().itertuples()):
    # if m.wyId == 2565632: print(m.wyId)
    # los jugadores titulares son los últimos 11 elementos de los arrays 'players_id'
    lineup_h = players_mins_DF.loc[m.wyId, m.teamId_home].players_id[-11:]
    lineup_a = players_mins_DF.loc[m.wyId, m.teamId_away].players_id[-11:]

    for player in lineup_h:
        for part in partidos_h[m.wyId]:
            # if part == 2565632: print(player)
            minsH[i] += players_match_mins[player].get(part,0)

    for player in lineup_a:
        for part in partidos_a[m.wyId]:
            minsA[i] += players_match_mins[player].get(part,0)

    # minsH = minsH / (npj*90)
    # minsA = minsA / (npj*90)

    # OPCION CODIGO
    # crear un diccionario grande de los ultimos N partidos (playerId : player_min) y hacer ReduceByKey(add), con lo cual tendremos la suma de minutos de cada jugador
    # a continuación escogeriamos solamente las Keys de los jugadores del lineup y las sumariamos
    

assert len(minsA) == len(minsH) == len(matches_wUltPartidos)

In [ ]:
matches_wUltPartidos['mins4_H'] = minsH / (npj*11)
matches_wUltPartidos['mins4_A'] = minsA / (npj*11)

In [ ]:
matches_wUltPartidos.mins4_H[2565632]

#### Añadir vectores de las demás caracteristicas

In [ ]:
def filterDB(db,matches,players):
    mask_players = False
    for p in players:
        mask_players = mask_players | np.array(db.playerId == p)

    mask_matches = False
    for m in matches:
        mask_matches = mask_matches | np.array(db.matchId == m)

    mask = mask_matches & mask_players
    
    return db[mask]

In [ ]:
# creamos feature de pases y disparos segun el lineup-11

shots_11H, shots_acc_11H, goals_H, goals_ratio_H = [], [], [], []
shots_11A, shots_acc_11A, goals_A, goals_ratio_A = [], [], [], []

passes_11H, passes_acc_11H, keyPasses_H = [], [], []
passes_11A, passes_acc_11A, keyPasses_A = [], [], []

for m in matches_wUltPartidos.reset_index().itertuples():
    #shots
    db_filtered_h = filterDB(shots,partidos_h[m.wyId],players_mins_DF.loc[(m.wyId,m.teamId_home)].players_id[-11:])
    shots_11H.append(len(db_filtered_h))
    shots_acc_11H.append(np.mean(db_filtered_h.accuracy))
    goals_H.append(sum(db_filtered_h.goal))
    db_filtered_a = filterDB(shots,partidos_a[m.wyId],players_mins_DF.loc[m.wyId].loc[m.teamId_away].players_id[-11:])
    shots_11A.append(len(db_filtered_a))
    shots_acc_11A.append(np.mean(db_filtered_a.accuracy))
    goals_A.append(sum(db_filtered_a.goal))
    # interesante el ratio de goles del line-up (goles lineup/golestotales)
    golestotales_h = sum([matches.loc[mh].score_home for mh in partidos_h[m.wyId]])
    golestotales_a = sum([matches.loc[ma].score_away for ma in partidos_a[m.wyId]])
    if golestotales_h == 0: goals_ratio_H.append(0)
    else: goals_ratio_H.append(sum(db_filtered_h.goal) / golestotales_h)
    if golestotales_a == 0: goals_ratio_A.append(0)
    else: goals_ratio_A.append(sum(db_filtered_a.goal) / golestotales_a)

    #passes
    db_filtered_h = filterDB(passes,partidos_h[m.wyId],players_mins_DF.loc[m.wyId].loc[m.teamId_home].players_id[-11:])
    passes_11H.append(len(db_filtered_h))
    passes_acc_11H.append(np.mean(db_filtered_h.accuracy))
    keyPasses_H.append(sum(db_filtered_h.keyPass))
    db_filtered_a = filterDB(passes,partidos_a[m.wyId],players_mins_DF.loc[m.wyId].loc[m.teamId_away].players_id[-11:])
    passes_11A.append(len(db_filtered_a))
    passes_acc_11A.append(np.mean(db_filtered_a.accuracy))
    keyPasses_A.append(sum(db_filtered_a.keyPass))

In [ ]:
assert len(shots_11H) == len(shots_acc_11H) == len(goals_ratio_H) == len(goals_ratio_A)

In [ ]:
matches_wUltPartidos['shots_11H']       = shots_11H
matches_wUltPartidos['shots_11A']       = shots_11A
matches_wUltPartidos['shots_acc_11H']   = shots_acc_11H
matches_wUltPartidos['shots_acc_11A']   = shots_acc_11A
matches_wUltPartidos['goals_H']         = goals_H
matches_wUltPartidos['goals_A']         = goals_A
matches_wUltPartidos['goals_ratio_H']   = goals_ratio_H
matches_wUltPartidos['goals_ratio_A']   = goals_ratio_A

matches_wUltPartidos['passes_11H']      = passes_11H
matches_wUltPartidos['passes_11A']      = passes_11A
matches_wUltPartidos['passes_acc_11H']  = passes_acc_11H
matches_wUltPartidos['passes_acc_11A']  = passes_acc_11A
matches_wUltPartidos['keyPasses_H']     = keyPasses_H
matches_wUltPartidos['keyPasses_A']     = keyPasses_A

In [ ]:
matches_wUltPartidos.columns

In [ ]:
# matches_wUltPartidos.to_csv(path_rawdata+'matches_wUltPartidos.csv',sep=';')
matches_wUltPartidos = pd.read_csv(path_rawdata+'matches_wUltPartidos.csv',sep=';',index_col='wyId')
matches_wUltPartidos.head(3)

#### Indicadores Ataque-Defensa

In [ ]:
ataque_defensa = [[[],[]],[[],[]]]
w = 1/np.arange(1,5)

for match in matches_wUltPartidos.reset_index().itertuples(): 
    df_ult_partH = matches.loc[partidos_h[match.wyId]][['teamId_home','teamId_away','score_away','score_home']]
    df_ult_partA = matches.loc[partidos_a[match.wyId]][['teamId_home','teamId_away','score_away','score_home']]

    df_ult = [df_ult_partH, df_ult_partA]
    col    = ['teamId_home','teamId_away']
    score  = ['score_away','score_home']

    for i,df in enumerate(df_ult):
        # avg_encajados, avg_marcados = np.mean(df[score[i]]), np.mean(df[score[not(i)]])
        avg_encajados = np.average(df[score[i]],weights=w), # damos +importancia a los partidos mas recientes
        avg_marcados  = np.average(df[score[not(i)]],weights=w)
        ataque_defensa[i][0].append(avg_encajados), ataque_defensa[i][1].append(avg_marcados)

# un equipo que solo ha recibido un gol y ha marcado X goles recibirá mejor puntuación
# que otro equipo que ha marcado los mismos goles pero no ha encajado ni un gol.
# si dividimos por la SUMA de los goles encajados desaparece este problema.
# sin embargo, un equipo que recibe un solo gol y no marca ninguno, 
# recibe menos puntuacion que otro que marca un gol y recibe muchos.

In [ ]:
assert len(ataque_defensa[0][0]) == len(matches_wUltPartidos)

In [ ]:
ataque_h  = np.array(ataque_defensa[0][1])
defensa_h = np.array(ataque_defensa[0][0])
ataque_a  = np.array(ataque_defensa[1][1])
defensa_a = np.array(ataque_defensa[1][0])

matches_wUltPartidos['ataque_h'], matches_wUltPartidos['defensa_h'] = ataque_h, defensa_h
matches_wUltPartidos['ataque_a'], matches_wUltPartidos['defensa_a'] = ataque_a, defensa_a

In [ ]:
avg_encajados_home = matches_wUltPartidos.groupby('teamId_home').agg(encajados=('score_away','mean'))
avg_marcados_home  = matches_wUltPartidos.groupby('teamId_home').agg(marcados=('score_home','mean'))
avg_encajados_away = matches_wUltPartidos.groupby('teamId_away').agg(encajados=('score_home','mean'))
avg_marcados_away  = matches_wUltPartidos.groupby('teamId_away').agg(marcados=('score_away','mean'))

avg_encajados = (pd.concat([avg_encajados_away,avg_encajados_home],axis=1)).mean(axis=1)  # (encajados como locales + commo encajados visit) / 2 -> pq son mismo # partidos
avg_marcados  = (pd.concat([avg_marcados_home,avg_marcados_away],axis=1)).mean(axis=1)

# comprimimos ambos en uno solo
avg_encajados_inv = 1/avg_encajados
indicador = np.mean([avg_marcados,avg_encajados_inv],axis=0)
indicador = pd.DataFrame({'indicador':indicador},index=avg_encajados.index)

# ataque_defensa_teams = pd.DataFrame({'avg_marcados':avg_marcados, 'avg_encajados':avg_encajados}).reset_index()
# teamId = ataque_defensa_teams['index']
# ataque_defensa_teams['teamId'] = teamId
# ataque_defensa_teams.drop('index',axis=1,inplace=True)


In [ ]:
# ataque_defensa_teams.set_index('teamId',inplace=True)
# indicador.to_csv(path_rawdata+'indicador_ataquedefensa_teams.csv',sep=';')
indicador

In [ ]:
indicador.sort_values(by='indicador',ascending=False)

In [ ]:
matches_wUltPartidos['ataquedefensa_H'] = matches_wUltPartidos.teamId_home.map(dict(indicador.indicador))
matches_wUltPartidos['ataquedefensa_A'] = matches_wUltPartidos.teamId_away.map(dict(indicador.indicador))

In [ ]:
matches_wUltPartidos.head(3)

In [ ]:
matches_wUltPartidos.to_csv(path_rawdata+'matches_wUltPartidos.csv',sep=';')